## 원본 데이터로부터 모델 학습에 사용할 수 있는 데이터를 추출해보자.

프로세싱 순서는 아래와 같다.

1. 사용할 수 없는 데이터는 제외한 다음 상위 7개의 state를 가지는 음성만을 가져온다. 

2. sample rate를 16000으로 통일한다.

3. 파일의 이름을 통해 state를 설정하기 위해 파일 이름을 모두 변경한다.

4. 음성 파일 앞뒤의 화이트 노이즈(또는 특정 임계값 이하의 에너지를 가지는 시점)를 제거한다.

5. 음성이 5초 단위로 분할하여 모두 동일한 길이를 가지도록 조정한다.

6. 음성의 노이즈를 제거한다.

7. 음성의 유사도를 분석하여 가장 유사한 n개의 데이터만 사용한다.

8. 모델의 학습 데이터로 사용한다.

In [7]:
# Load packages
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm

In [8]:
# Set Path
main_path = os.path.join(os.getcwd().rsplit(
    'baby-cry-classification')[0], 'baby-cry-classification')
data_path = os.path.join(main_path, 'data2')
csv_path = os.path.join(main_path, 'origin_data_info.csv')
origin_data_path = os.path.join(main_path, 'origin_data')
sample_data_path = os.path.join(data_path, 'sample_data')

import sys
sys.path.append(main_path)

In [9]:
# Load custon packages
from utils.os import *
from utils.sound import *

from trans_data import (get_state_list_from_dir_name,
                        create_state_folder,
                        rename_files_by_state,
                        trim_audio,
                        split_audio_list_in_sec,
                        reduced_base_noise)

In [10]:
# 1. 사용할 수 없는 데이터는 제외한 다음 상위 7개의 state를 가지는 음성만을 가져온다. 
# 2. sample rate를 16000으로 통일한다.
create_state_folder(origin_data_path, csv_path, data_path)

전체 3688개의 데이터 중 사용할 수 없는 216개의 데이터가 제외되었다.


In [11]:
# 3. 파일의 이름을 통해 state를 설정하기 위해 파일 이름을 모두 변경한다.
file_list = rename_files_by_state(data_path)

In [12]:
# 4. 음성 파일 앞뒤의 화이트 노이즈(또는 특정 임계값 이하의 에너지를 가지는 시점)를 제거한다.
failed_file_list = []
for i in tqdm(range(len(file_list))):
    try:
        trim_audio(file_list[i], inplace=True)
    except:
        failed_file_list.append(file_list[i])

print(f'trimming에 실패한 {len(failed_file_list)}개의 파일이 삭제됨.')
for file in failed_file_list:
    file_list.remove(file)
    remove_file(file)

  0%|          | 0/3238 [00:00<?, ?it/s]

100%|██████████| 3238/3238 [23:36<00:00,  2.29it/s]

trimming에 실패한 5개의 파일이 삭제됨.


In [13]:
# 5. 음성이 2초 단위로 분할하여 모두 동일한 길이를 가지도록 조정한다.
split_audio_list = split_audio_list_in_sec(file_list, 1.5, inplace=True)

# renaming
file_list = rename_files_by_state(data_path)

In [23]:
# 데이터프레임을 생성하여 파일을 분석해 유사도분석을 통해 제거할 음성의 개수를 정하자.
df = pd.DataFrame({'file':file_list})
df['duration'] = df['file'].apply(lambda file: get_duration(file))
df['file'] = df['file'].apply(lambda file: file.rsplit('/', 1)[1])
df['state'] = df['file'].apply(lambda file: file.split('_', 1)[0]).astype('category')

# 모든 파일들은 2초의 값을 가짐을 확인할 수 있다.
count_by_state = df[['file', 'state']].groupby('state').count().reset_index().sort_values('file').reset_index(drop=True)
print(df['duration'].value_counts())
print(f'\n{count_by_state.state[0]}이 {count_by_state.file[0]}로 가장 적게 존재하며 {count_by_state.state[len(count_by_state) - 1]}가 {count_by_state.file[len(count_by_state) - 1]}로 가장 많이 존재한다.')
count_by_state

2.0    17201
Name: duration, dtype: int64

diaper이 1737로 가장 적게 존재하며 sad가 3915로 가장 많이 존재한다.


,state,file
0,diaper,1737
1,uncomfortable,1846
2,sleepy,1991
3,awake,2030
4,hug,2435
5,hungry,3247
6,sad,3915


In [29]:
# 6. 음성의 노이즈를 제거한다.
for i in tqdm(range(len(file_list))):
    reduced_base_noise(file_list[i], inplace=True)

100%|██████████| 17201/17201 [04:06<00:00, 69.87it/s]


In [30]:
min_file_count = count_by_state.file[0]
use_file_count = round(min_file_count - (min_file_count / 10))
print(f'가장 적은 state의 파일 개수: {min_file_count}')
print(f'유사도 분석을 통해 사용할 상위 90%의 파일의 개수: {use_file_count}')

가장 적은 state의 파일 개수: 1737
유사도 분석을 통해 사용할 상위 90%의 파일의 개수: 1563


In [32]:
# 7. 음성의 유사도를 분석하여 가장 유사한 n개의 데이터만 사용한다.

def delete_dissimilar_wavs(data_path:str, n_limit:int):
  state_list = ['sad', 'hug', 'diaper', 'hungry', 'sleepy', 'awake', 'uncomfortable']
  for state in state_list:
      state_path = os.path.join(data_path, state)
      state_file_list = [os.path.join(state_path, file) for file in os.listdir(state_path)]

      # get similarities
      sim_file_list = get_similarities(state_file_list)

      # 상위 90%를 제외하고 모두 제거한다.
      del_file_list = sim_file_list[n_limit:]
      for file in del_file_list:
          remove_file(file)
      print(f'State {state} Done.\n')

delete_dissimilar_wavs(data_path, use_file_count)

Processing: 100%|██████████| 3915/3915 [23:41<00:00,  2.75it/s]


State sad Done.



Processing: 100%|██████████| 2435/2435 [09:09<00:00,  4.43it/s]


State hug Done.



Processing: 100%|██████████| 1737/1737 [04:41<00:00,  6.18it/s]


...

State sleepy Done.



Processing: 100%|██████████| 2030/2030 [06:24<00:00,  5.28it/s]


State awake Done.



Processing: 100%|██████████| 1846/1846 [05:17<00:00,  5.82it/s]


State uncomfortable Done.



In [33]:
_ = rename_files_by_state(data_path)